In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
import pandas_ta as ta

# Se requieren las librerías pandas, matplotlib, yfinance, numpy y pandas_ta.
# Puedes instalarlas con: !pip install pandas matplotlib yfinance numpy pandas_ta

def fetch_data(ticker, period="1y"):
    """
    Descarga datos históricos de precios de un ticker.
    Puedes encontrar tickers de acciones, criptomonedas y otros activos en sitios como Yahoo Finance.
    
    Por ejemplo:
    - Acciones: AAPL (Apple), GOOG (Alphabet)
    - Criptomonedas: BTC-USD (Bitcoin), ETH-USD (Ethereum)
    - Índices: ^GSPC (S&P 500)
    
    Args:
        ticker (str): Símbolo de la acción o criptomoneda.
        period (str): Período de tiempo para descargar los datos (ej. "1y", "5y", "max").
    
    Returns:
        pd.DataFrame: DataFrame con los datos de precios o None si falla.
    """
    try:
        data = yf.download(ticker, period=period)
        if data.empty:
            print(f"Error: No se pudieron descargar datos para el ticker '{ticker}'.")
            return None
        return data
    except Exception as e:
        print(f"Ocurrió un error al descargar los datos: {e}")
        return None

def calculate_metrics(trades, initial_capital, final_value):
    """Calcula y muestra un resumen del rendimiento de la estrategia."""
    if not trades:
        print("No se realizaron operaciones de trading.")
        total_profit = final_value - initial_capital
        total_return = (total_profit / initial_capital) * 100 if initial_capital > 0 else 0
        
        print("\n--- Resumen de la Estrategia ---")
        print(f"Capital Inicial: ${initial_capital:,.2f}")
        print(f"Valor Final del Portafolio: ${final_value:,.2f}")
        print(f"Ganancia/Pérdida Neta: ${total_profit:,.2f}")
        print(f"Rendimiento Total: {total_return:.2f}%")
        print("No se encontraron operaciones de compra/venta.")
        return total_return, total_profit

    # Cálculos generales
    total_trades = len(trades)
    winning_trades = sum(1 for trade in trades if trade['profit'] > 0)
    losing_trades = total_trades - winning_trades
    win_rate = (winning_trades / total_trades) * 100 if total_trades > 0 else 0

    # Cálculos de ganancias y pérdidas
    total_profit = sum(trade['profit'] for trade in trades)
    total_return = (total_profit / initial_capital) * 100 if initial_capital > 0 else 0
    avg_profit = total_profit / total_trades if total_trades > 0 else 0
    
    if winning_trades > 0:
        avg_win = sum(trade['profit'] for trade in trades if trade['profit'] > 0) / winning_trades
    else:
        avg_win = 0

    if losing_trades > 0:
        avg_loss = sum(trade['profit'] for trade in trades if trade['profit'] < 0) / losing_trades
    else:
        avg_loss = 0
    
    print("\n--- Resumen de la Estrategia ---")
    print(f"Capital Inicial: ${initial_capital:,.2f}")
    print(f"Valor Final del Portafolio: ${final_value:,.2f}")
    print(f"Ganancia/Pérdida Neta: ${total_profit:,.2f}")
    print(f"Rendimiento Total: {total_return:.2f}%")
    print(f"Total de Operaciones: {total_trades}")
    print(f"Operaciones Ganadoras: {winning_trades}")
    print(f"Operaciones Perdedoras: {losing_trades}")
    print(f"Tasa de Éxito (Win Rate): {win_rate:.2f}%")
    print(f"Ganancia Promedio por Operación: ${avg_profit:.2f}")
    print(f"Ganancia Promedio en Operaciones Ganadoras: ${avg_win:.2f}")
    print(f"Pérdida Promedio en Operaciones Perdedoras: ${avg_loss:.2f}")
    return total_return, total_profit

def save_to_csv(data, portfolio, filename="backtest_report.csv"):
    """Guarda el DataFrame de resultados en un archivo CSV."""
    full_report = pd.concat([data, portfolio], axis=1)
    full_report.to_csv(filename)
    print(f"El informe se ha guardado exitosamente en '{filename}'.")

def run_backtest(data, strategy_name, **kwargs):
    """
    Ejecuta un backtest basado en una estrategia seleccionada.

    Args:
        data (pd.DataFrame): DataFrame con los datos de precios.
        strategy_name (str): Nombre de la estrategia a usar.
        **kwargs: Parámetros específicos de la estrategia.
    """
    initial_capital = 10000
    portfolio_data = []
    trades = []
    
    in_position = False
    shares = 0
    cash = initial_capital
    entry_price = 0
    
    # Creación de la columna de señal de trading
    data['Signal'] = 0.0
    
    if strategy_name == "SMA":
        short_window = kwargs.get('short_window', 50)
        long_window = kwargs.get('long_window', 200)
        data['Short_MA'] = data['Close'].rolling(window=short_window).mean()
        data['Long_MA'] = data['Close'].rolling(window=long_window).mean()
        data = data.dropna()
        # Generar señales de compra (1.0) y venta (-1.0)
        data['Signal'] = np.where(data['Short_MA'] > data['Long_MA'], 1.0, 0.0)
        data['Signal'] = np.where(data['Short_MA'] < data['Long_MA'], -1.0, data['Signal'])
    
    elif strategy_name == "BB":
        window = kwargs.get('window', 20)
        num_std = kwargs.get('num_std', 2)
        data['SMA'] = data['Close'].rolling(window=window).mean()
        data['StdDev'] = data['Close'].rolling(window=window).std()
        data['UpperBand'] = data['SMA'] + (data['StdDev'] * num_std)
        data['LowerBand'] = data['SMA'] - (data['StdDev'] * num_std)
        data = data.dropna()
        data['Signal'] = np.where(data['Close'] < data['LowerBand'], 1.0, 0.0)
        data['Signal'] = np.where(data['Close'] > data['UpperBand'], -1.0, data['Signal'])

    elif strategy_name == "RSI":
        period = kwargs.get('period', 14)
        overbought = kwargs.get('overbought', 70)
        oversold = kwargs.get('oversold', 30)
        delta = data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        data['RSI'] = 100 - (100 / (1 + rs))
        data = data.dropna()
        data['Signal'] = np.where(data['RSI'] < oversold, 1.0, 0.0)
        data['Signal'] = np.where(data['RSI'] > overbought, -1.0, data['Signal'])
        
    elif strategy_name == "MACD":
        fast_period = kwargs.get('fast_period', 12)
        slow_period = kwargs.get('slow_period', 26)
        signal_period = kwargs.get('signal_period', 9)
        exp1 = data['Close'].ewm(span=fast_period, adjust=False).mean()
        exp2 = data['Close'].ewm(span=slow_period, adjust=False).mean()
        data['MACD'] = exp1 - exp2
        data['Signal_Line'] = data['MACD'].ewm(span=signal_period, adjust=False).mean()
        data = data.dropna()
        data['Signal'] = np.where(data['MACD'] > data['Signal_Line'], 1.0, 0.0)
        data['Signal'] = np.where(data['MACD'] < data['Signal_Line'], -1.0, data['Signal'])

    elif strategy_name == "OBV":
        data['OBV'] = np.where(data['Close'] > data['Close'].shift(1), data['Volume'],
                               np.where(data['Close'] < data['Close'].shift(1), -data['Volume'], 0)).cumsum()
        data['OBV_SMA'] = data['OBV'].rolling(window=20).mean()
        data = data.dropna()
        data['Signal'] = np.where(data['OBV'] > data['OBV_SMA'], 1.0, 0.0)
        data['Signal'] = np.where(data['OBV'] < data['OBV_SMA'], -1.0, data['Signal'])
    
    elif strategy_name == "AO":
        # Se requiere un mínimo de 34 períodos para este cálculo
        if len(data) < 34:
            print("Error: Los datos son insuficientes para calcular el Awesome Oscillator. Por favor, selecciona un período más largo.")
            return
            
        short_ma = data['Close'].rolling(window=5).mean()
        long_ma = data['Close'].rolling(window=34).mean()
        data['AO'] = short_ma - long_ma
        data = data.dropna()
        data['Signal'] = np.where(data['AO'] > 0, 1.0, 0.0)
        data['Signal'] = np.where(data['AO'] < 0, -1.0, data['Signal'])

    elif strategy_name == "Stochastic":
        k_period = kwargs.get('k_period', 14)
        d_period = kwargs.get('d_period', 3)
        
        # Calcular el mínimo bajo y el máximo alto del período
        low_min = data['Low'].rolling(window=k_period).min()
        high_max = data['High'].rolling(window=k_period).max()
        
        # Usar operaciones de Series de Pandas, más robustas
        numerator = data['Close'] - low_min
        denominator = high_max - low_min
        
        # Calcular %K, manejando la división por cero
        data['%K'] = (100 * numerator / denominator).fillna(0)
        
        # Calcular %D como la media móvil de %K
        data['%D'] = data['%K'].rolling(window=d_period).mean()
        data = data.dropna()
        
        # Señales de compra/venta
        data.loc[(data['%K'].shift(1) < data['%D'].shift(1)) & (data['%K'] > data['%D']), 'Signal'] = 1.0
        data.loc[(data['%K'].shift(1) > data['%D'].shift(1)) & (data['%K'] < data['%D']), 'Signal'] = -1.0
    
    elif strategy_name == "SAR":
        # Reemplazamos la compleja implementación manual por la librería pandas_ta
        try:
            # Esta función de la librería añade las columnas SAR, SART y AF al DataFrame
            data.ta.sar(append=True)
            data = data.dropna()
            
            # Las señales se basan en el cambio de tendencia (SART)
            data.loc[(data['SART'].shift(1) == -1) & (data['SART'] == 1), 'Signal'] = 1.0  # Señal de compra
            data.loc[(data['SART'].shift(1) == 1) & (data['SART'] == -1), 'Signal'] = -1.0 # Señal de venta
        except Exception as e:
            print(f"Error al calcular el SAR Parabólico con pandas_ta: {e}")
            print("Asegúrate de haber instalado la librería con 'pip install pandas-ta'")
            return
            
    # Verificar si aún hay datos después de la limpieza
    if data.empty:
        print("Error: Los datos son insuficientes para ejecutar la estrategia. Por favor, selecciona un período más largo.")
        return
    
    # Simular las operaciones de manera más robusta
    for i in range(1, len(data)):
        if data['Signal'].iloc[i] == 1 and not in_position:  # Señal de compra
            shares = cash / data['Close'].iloc[i]
            cash = 0.0
            in_position = True
            entry_price = data['Close'].iloc[i]
            
        elif data['Signal'].iloc[i] == -1 and in_position:  # Señal de venta
            cash = shares * data['Close'].iloc[i]
            exit_price = data['Close'].iloc[i]
            profit = (exit_price - entry_price) * shares
            trades.append({'entry_date': data.index[i-1], 'exit_date': data.index[i], 'profit': float(profit)})
            shares = 0.0
            in_position = False
        
        # Guardar el estado del portafolio para cada día
        total_value = cash + shares * data['Close'].iloc[i]
        portfolio_data.append({'Date': data.index[i], 'Cash': cash, 'Shares': shares, 'Total': total_value})
    
    portfolio = pd.DataFrame(portfolio_data).set_index('Date')
    
    # Manejar si el backtest termina con una posición abierta
    final_value = portfolio['Total'].iloc[-1].item() if not portfolio.empty else float(initial_capital)

    # Calcular y mostrar métricas
    calculate_metrics(trades, initial_capital, final_value)
    
    # Graficar los resultados
    plt.style.use('dark_background')
    fig, ax1 = plt.subplots(figsize=(12, 8))
    
    color = 'tab:blue'
    ax1.set_xlabel('Fecha')
    ax1.set_ylabel('Precio', color=color)
    ax1.plot(data.index, data['Close'], label='Precio de cierre', color='white', linewidth=1)
    ax1.tick_params(axis='y', labelcolor=color)
    
    if strategy_name == "SMA":
        ax1.plot(data.index, data['Short_MA'], label=f'Media móvil {kwargs.get("short_window", 50)} días', color='cyan')
        ax1.plot(data.index, data['Long_MA'], label=f'Media móvil {kwargs.get("long_window", 200)} días', color='orange')
    elif strategy_name == "BB":
        ax1.plot(data.index, data['SMA'], label=f'Media móvil {kwargs.get("window", 20)} días', color='yellow')
        ax1.plot(data.index, data['UpperBand'], label='Banda Superior', color='red', linestyle='--')
        ax1.plot(data.index, data['LowerBand'], label='Banda Inferior', color='green', linestyle='--')
    elif strategy_name == "SAR":
        if 'SAR' in data.columns:
            ax1.plot(data.index, data['SAR'], 'o', markersize=2, label='SAR Parabólico', color='yellow')
    
    # Marcar las señales de compra y venta
    buy_signals = data.loc[data['Signal'] == 1.0]
    sell_signals = data.loc[data['Signal'] == -1.0]

    ax1.plot(buy_signals.index, buy_signals['Close'], '^', markersize=10, color='lime', label='Señal de compra')
    ax1.plot(sell_signals.index, sell_signals['Close'], 'v', markersize=10, color='red', label='Señal de venta')
    
    ax2 = ax1.twinx()
    color = 'tab:green'
    ax2.set_ylabel('Valor del Portafolio', color=color)
    if not portfolio.empty:
      ax2.plot(portfolio.index, portfolio['Total'], label='Valor del Portafolio', color=color, linestyle='-')
    ax2.tick_params(axis='y', labelcolor=color)
    
    fig.tight_layout()
    plt.title(f'Backtest de Estrategia: {strategy_name}')
    fig.legend(loc="upper left", bbox_to_anchor=(0.1,0.9))
    plt.grid(True)
    plt.show()

    # Gráficos adicionales para indicadores
    if strategy_name == "RSI":
        plt.style.use('dark_background')
        plt.figure(figsize=(12, 4))
        plt.plot(data.index, data['RSI'], label='RSI', color='purple')
        plt.axhline(y=kwargs.get("overbought", 70), color='red', linestyle='--', label='Sobrecompra')
        plt.axhline(y=kwargs.get("oversold", 30), color='green', linestyle='--', label='Sobrevendido')
        plt.title('Índice de Fuerza Relativa (RSI)')
        plt.xlabel('Fecha')
        plt.ylabel('Valor del RSI')
        plt.legend()
        plt.grid(True)
        plt.show()

    if strategy_name == "MACD":
        plt.style.use('dark_background')
        plt.figure(figsize=(12, 4))
        plt.plot(data.index, data['MACD'], label='MACD', color='blue')
        plt.plot(data.index, data['Signal_Line'], label='Línea de Señal', color='red')
        plt.title('Convergencia/Divergencia de Media Móvil (MACD)')
        plt.xlabel('Fecha')
        plt.ylabel('Valor de MACD')
        plt.legend()
        plt.grid(True)
        plt.show()

    if strategy_name == "OBV":
        plt.style.use('dark_background')
        plt.figure(figsize=(12, 4))
        plt.plot(data.index, data['OBV'], label='OBV', color='orange')
        plt.plot(data.index, data['OBV_SMA'], label='SMA de OBV', color='purple', linestyle='--')
        plt.title('On-Balance Volume (OBV)')
        plt.xlabel('Fecha')
        plt.ylabel('Valor de OBV')
        plt.legend()
        plt.grid(True)
        plt.show()
    
    if strategy_name == "AO":
        plt.style.use('dark_background')
        plt.figure(figsize=(12, 4))
        # Se asegura de que el DataFrame no esté vacío antes de graficar
        if not data.empty:
            plt.bar(data.index, data['AO'], label='Awesome Oscillator', color=np.where(data['AO'] >= 0, 'lime', 'red'))
        plt.axhline(y=0, color='white', linestyle='--')
        plt.title('Awesome Oscillator (AO)')
        plt.xlabel('Fecha')
        plt.ylabel('Valor de AO')
        plt.legend()
        plt.grid(True)
        plt.show()

    if strategy_name == "Stochastic":
        plt.style.use('dark_background')
        plt.figure(figsize=(12, 4))
        plt.plot(data.index, data['%K'], label='%K', color='lime')
        plt.plot(data.index, data['%D'], label='%D', color='red')
        plt.axhline(y=80, color='red', linestyle='--', label='Sobrecompra')
        plt.axhline(y=20, color='green', linestyle='--', label='Sobrevendido')
        plt.title('Oscilador Estocástico')
        plt.xlabel('Fecha')
        plt.ylabel('Valor')
        plt.legend()
        plt.grid(True)
        plt.show()


    # Opción para descargar
    while True:
        download_choice = input("¿Deseas descargar el informe de backtest como CSV? (s/n): ").lower()
        if download_choice == 's':
            save_to_csv(data, portfolio)
            break
        elif download_choice == 'n':
            break
        else:
            print("Selección no válida.")

def main():
    """Función principal que maneja la interacción con el usuario."""
    
    # 1. Menú de idioma
    while True:
        lang_choice = input("Select a language / Selecciona un idioma (en/es): ").lower()
        if lang_choice == 'en':
            print("You have selected English.")
            break
        elif lang_choice == 'es':
            print("Has seleccionado español.")
            break
        else:
            print("Invalid selection. Please choose 'en' or 'es'.")
            
    # 2. Preguntar por el activo
    if lang_choice == 'es':
        print("Puedes encontrar tickers en sitios como Yahoo Finance. ")
        ticker = input("Ingresa el Ticker (ej. BTC-USD, AAPL): ")
        period = input("Ingresa el período de tiempo (ej. 1y, 5y, max): ")
    else:
        print("You can find tickers on websites like Yahoo Finance. ")
        ticker = input("Enter Ticker (e.g. BTC-USD, AAPL): ")
        period = input("Enter time period (e.g. 1y, 5y, max): ")

    # Descargar los datos
    data = fetch_data(ticker, period)
    if data is None:
        return

    # 3. Lista de estrategias
    if lang_choice == 'es':
        print("\n--- Estrategias disponibles ---")
        print("1. Cruce de Medias Móviles (SMA)")
        print("2. Bandas de Bollinger (BB)")
        print("3. Índice de Fuerza Relativa (RSI)")
        print("4. Convergencia/Divergencia de Media Móvil (MACD)")
        print("5. On-Balance Volume (OBV)")
        print("6. Awesome Oscillator (AO)")
        print("7. Oscilador Estocástico")
        print("8. SAR Parabólico")
        strategy_choice = input("Selecciona un número de estrategia: ")
    else:
        print("\n--- Available Strategies ---")
        print("1. Moving Average Crossover (SMA)")
        print("2. Bollinger Bands (BB)")
        print("3. Relative Strength Index (RSI)")
        print("4. Moving Average Convergence Divergence (MACD)")
        print("5. On-Balance Volume (OBV)")
        print("6. Awesome Oscillator (AO)")
        print("7. Stochastic Oscillator")
        print("8. Parabolic SAR")
        strategy_choice = input("Select a strategy number: ")

    # 4. Ejecutar el backtest según la estrategia seleccionada
    if strategy_choice == '1':
        if lang_choice == 'es':
            mod_choice = input("¿Quieres usar los valores predeterminados (50/200)? (s/n): ").lower()
            if mod_choice == 's':
                run_backtest(data, "SMA")
            elif mod_choice == 'n':
                short_window = int(input("Ingresa el período para la media móvil corta: "))
                long_window = int(input("Ingresa el período para la media móvil larga: "))
                run_backtest(data, "SMA", short_window=short_window, long_window=long_window)
        else:
            mod_choice = input("Do you want to use the default values (50/200)? (y/n): ").lower()
            if mod_choice == 'y':
                run_backtest(data, "SMA")
            elif mod_choice == 'n':
                short_window = int(input("Enter the period for the short moving average: "))
                long_window = int(input("Enter the period for the long moving average: "))
                run_backtest(data, "SMA", short_window=short_window, long_window=long_window)

    elif strategy_choice == '2':
        if lang_choice == 'es':
            mod_choice = input("¿Quieres usar los valores predeterminados (20, 2)? (s/n): ").lower()
            if mod_choice == 's':
                run_backtest(data, "BB")
            elif mod_choice == 'n':
                window = int(input("Ingresa el período: "))
                num_std = float(input("Ingresa el número de desviaciones estándar: "))
                run_backtest(data, "BB", window=window, num_std=num_std)
        else:
            mod_choice = input("Do you want to use the default values (20, 2)? (y/n): ").lower()
            if mod_choice == 'y':
                run_backtest(data, "BB")
            elif mod_choice == 'n':
                window = int(input("Enter the period: "))
                num_std = float(input("Enter the number of standard deviations: "))
                run_backtest(data, "BB", window=window, num_std=num_std)
    
    elif strategy_choice == '3':
        if lang_choice == 'es':
            mod_choice = input("¿Quieres usar los valores predeterminados (14, 70, 30)? (s/n): ").lower()
            if mod_choice == 's':
                run_backtest(data, "RSI")
            elif mod_choice == 'n':
                period = int(input("Ingresa el período (ej. 14): "))
                overbought = int(input("Ingresa el nivel de sobrecompra (ej. 70): "))
                oversold = int(input("Ingresa el nivel de sobreventa (ej. 30): "))
                run_backtest(data, "RSI", period=period, overbought=overbought, oversold=oversold)
        else:
            mod_choice = input("Do you want to use the default values (14, 70, 30)? (y/n): ").lower()
            if mod_choice == 'y':
                run_backtest(data, "RSI")
            elif mod_choice == 'n':
                period = int(input("Enter the period (e.g. 14): "))
                overbought = int(input("Enter the overbought level (e.g. 70): "))
                oversold = int(input("Enter the oversold level (e.g. 30): "))
                run_backtest(data, "RSI", period=period, overbought=overbought, oversold=oversold)

    elif strategy_choice == '4':
        if lang_choice == 'es':
            mod_choice = input("¿Quieres usar los valores predeterminados (12, 26, 9)? (s/n): ").lower()
            if mod_choice == 's':
                run_backtest(data, "MACD")
            elif mod_choice == 'n':
                fast_period = int(input("Ingresa el período rápido (ej. 12): "))
                slow_period = int(input("Ingresa el período lento (ej. 26): "))
                signal_period = int(input("Ingresa el período de la línea de señal (ej. 9): "))
                run_backtest(data, "MACD", fast_period=fast_period, slow_period=slow_period, signal_period=signal_period)
        else:
            mod_choice = input("Do you want to use the default values (12, 26, 9)? (y/n): ").lower()
            if mod_choice == 'y':
                run_backtest(data, "MACD")
            elif mod_choice == 'n':
                fast_period = int(input("Enter the fast period (e.g. 12): "))
                slow_period = int(input("Enter the slow period (e.g. 26): "))
                signal_period = int(input("Enter the signal line period (e.g. 9): "))
                run_backtest(data, "MACD", fast_period=fast_period, slow_period=slow_period, signal_period=signal_period)
    
    elif strategy_choice == '5':
        if lang_choice == 'es':
            mod_choice = input("¿Quieres usar el valor predeterminado (20)? (s/n): ").lower()
            if mod_choice == 's':
                run_backtest(data, "OBV")
            elif mod_choice == 'n':
                obv_window = int(input("Ingresa el período para la media móvil de OBV: "))
                run_backtest(data, "OBV", window=obv_window)
        else:
            mod_choice = input("Do you want to use the default value (20)? (y/n): ").lower()
            if mod_choice == 'y':
                run_backtest(data, "OBV")
            elif mod_choice == 'n':
                obv_window = int(input("Enter the period for the OBV moving average: "))
                run_backtest(data, "OBV", window=obv_window)

    elif strategy_choice == '6':
        if lang_choice == 'es':
            run_backtest(data, "AO")
        else:
            run_backtest(data, "AO")
    
    elif strategy_choice == '7':
        if lang_choice == 'es':
            mod_choice = input("¿Quieres usar los valores predeterminados (14, 3)? (s/n): ").lower()
            if mod_choice == 's':
                run_backtest(data, "Stochastic")
            elif mod_choice == 'n':
                k_period = int(input("Ingresa el período %K (ej. 14): "))
                d_period = int(input("Ingresa el período %D (ej. 3): "))
                run_backtest(data, "Stochastic", k_period=k_period, d_period=d_period)
        else:
            mod_choice = input("Do you want to use the default values (14, 3)? (y/n): ").lower()
            if mod_choice == 'y':
                run_backtest(data, "Stochastic")
            elif mod_choice == 'n':
                k_period = int(input("Enter the %K period (e.g. 14): "))
                d_period = int(input("Enter the %D period (e.g. 3): "))
                run_backtest(data, "Stochastic", k_period=k_period, d_period=d_period)
    
    elif strategy_choice == '8':
        if lang_choice == 'es':
            mod_choice = input("¿Quieres usar los valores predeterminados (0.02, 0.2)? (s/n): ").lower()
            if mod_choice == 's':
                run_backtest(data, "SAR")
            elif mod_choice == 'n':
                accel = float(input("Ingresa el factor de aceleración (ej. 0.02): "))
                max_accel = float(input("Ingresa el factor de aceleración máximo (ej. 0.2): "))
                run_backtest(data, "SAR", acceleration_factor=accel, max_acceleration_factor=max_accel)
        else:
            mod_choice = input("Do you want to use the default values (0.02, 0.2)? (y/n): ").lower()
            if mod_choice == 'y':
                run_backtest(data, "SAR")
            elif mod_choice == 'n':
                accel = float(input("Enter the acceleration factor (e.g. 0.02): "))
                max_accel = float(input("Enter the max acceleration factor (e.g. 0.2): "))
                run_backtest(data, "SAR", acceleration_factor=accel, max_acceleration_factor=max_accel)

    else:
        if lang_choice == 'es':
            print("Estrategia no válida. Por favor, reinicia el programa.")
        else:
            print("Invalid strategy. Please restart the program.")

if __name__ == "__main__":
    main()


Select a language / Selecciona un idioma (en/es):  es


Has seleccionado español.
Puedes encontrar tickers en sitios como Yahoo Finance. 


Ingresa el Ticker (ej. BTC-USD, AAPL):  aapl
Ingresa el período de tiempo (ej. 1y, 5y, max):  1y


C:\Users\josep\AppData\Local\Temp\ipykernel_22244\3341207894.py:28: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=period)
[*********************100%***********************]  1 of 1 completed



--- Estrategias disponibles ---
1. Cruce de Medias Móviles (SMA)
2. Bandas de Bollinger (BB)
3. Índice de Fuerza Relativa (RSI)
4. Convergencia/Divergencia de Media Móvil (MACD)
5. On-Balance Volume (OBV)
6. Awesome Oscillator (AO)
7. Oscilador Estocástico
8. SAR Parabólico


Selecciona un número de estrategia:  8
¿Quieres usar los valores predeterminados (0.02, 0.2)? (s/n):  s


Error al calcular el SAR Parabólico con pandas_ta: 'AnalysisIndicators' object has no attribute 'sar'
Asegúrate de haber instalado la librería con 'pip install pandas-ta'
